In [43]:
from scipy.spatial.distance import squareform, pdist
from pylab import *
from numpy import * 
from scipy.linalg import null_space,solve
from itertools import product, combinations

N      = 4                 # Dimension of crystal
Ndepth = 10                # Iterations to grow crystal

θ  = float32(linspace(0,pi,N+1)[1:]) # Arrange basis vectors evenly on [0,π)
A  = float32(array([cos(θ),sin(θ)])) # Convert from complex notation
Ai = float32(pinv(A))

# Directions in which to search
D = int32(concatenate([eye(N),-eye(N)])) 

# Matrix to rotate a lattice point π/N in 2D plane
S = zeros((N,N),'f')
S[ -1,0 ] = -1
S[:-1,1:] = eye(N-1)

# Random seed to nucleate crystal
# Set to zero to get a symmetric version
seed = tuple(int32(Ai@(10000000*random.randn(2))))
print('seed:',seed)

# Null space: collapses cutting plane to point at origin
C = null_space(A).T

# All vertices of an N-cube
O = array([*product(*([0,1],)*N)])-0.5

# All vertices of a 2-cube (used to generate N-2 sub-facets)
F = array([*product(*([0,1],)*2)])

# A single corner ("origin" vertex) of an N-2 cube
# plus N-2 vectors defining the other points on the N-2 cube
# Used to test if sub-facet contains the origins
i = [0]+[*2**arange(N-2)]
G = array([*product(*([0,1],)*(N-2))])[i,:]

# All possible pairs of dimension for generating
# all possible N-2 sub-facets of the N-cube
I = [*combinations(range(N),2)]

# Plot
def plotpoints(p):
    u  = Ai.T@p
    D  = squareform(pdist(p.T))
    e  = {tuple(sorted(e)) for e in zip(*where(abs(D-1)<1e-6))}
    xy = concatenate([[u[:,a],u[:,b],[NaN,NaN]] for a,b in e])
    plot(*xy.T,lw=0.5,color='k')
    tight_layout()
    axis("equal")
    axis("off")
    
# Given a list of N-cube vertices, construct a list
# of all N-2-cube sub-facets.
subfacets = []
for ij in I:
    # For every pair of directions, and for every 2-cube
    # in each pair of directions... Generate all possible
    # N-2-cube sub-facets.
    v0 = F@eye(N,dtype='f')[ij,:]
    v1 = G@eye(N,dtype='f')[[*{*range(N)}-{*ij}]]
    subfacets.extend(v0[:,None,:]+v1[None,:,:])
vxid = int32(subfacets)@2**arange(N,dtype='i')

# Chec if a sub-facet contains the origin
def subfacet_contains(v):
    p0 = v[0]
    vk = v[1:,:]-p0
    q  = -solve(vk@vk.T,vk@p0)
    return all(q>=0) and all(q<=1)
    
# Generic caching intersection check function
# Map all points to first quadrant and memoize
def cached_crystal_checker(check):
    # Cache results to save time (memoization)
    cache = {}
    def f(q):
        # Convert test point to immutable tuple for cache key
        q = int32(q)
        k = tuple(q) 
        if not k in cache:
            # Recompute intersection test if not in cache
            # Use symmetry: reduce tests to points in 1st sector
            h = angle([1.,1.j]@A@q)
            cache[k]=check(q) if 0<=h<=(pi/N*1.1) else f(S@q)
        return cache[k]
    f.__cache__ = cache
    return f


# Get slopes and spacing of boundary lines
m = 1/tan(θ[:-1])
d = 1/sin(θ[:-1])

@cached_crystal_checker
def check_pm(p,eps=1e-2,maxiter=1000):
    p = int32(p)
    # Evaluate upper/lower bounds
    lp,up = p[:-1]-0.5, p[:-1]+0.5
    def feasibility_surface(p,x):
        u,l = x*m+up*d, x*m+lp*d
        y   = np.max(l)-np.min(u)
        dy  = m[np.argmax(l)]-m[np.argmin(u)]
        return y,dy
    # Initial bounds and feasability checks
    # If either bound satisfies, success
    # If better point not in bounds, fail
    x0,x1  = p[-1]-0.5, p[-1]+0.5
    y0,dy0 = feasibility_surface(p,x0)
    y1,dy1 = feasibility_surface(p,x1)
    if y0 < 0 or y1 < 0: return True
    if dy0>=0 or dy1<=0: return False
    for i in range(maxiter):
        # Guess min by following slope at bounds
        xm = (y1-y0-dy1*x1+dy0*x0)/(dy0-dy1)
        assert xm>=x0-eps
        assert xm<=x1+eps
        # Midpoint works? still feasible? 
        ym,dym = feasibility_surface(p,xm)
        if ym<0:   return True
        if dym==0: return False
        # Binary search: move inward if not done
        if dym<0:
            # If better point not in bounds, fail
            if dym>=0 or abs(x0-xm)<eps: return False
            x0,y0,dy0 = xm,ym,dym
        else:
            if dym<=0 or abs(x1-xm)<eps: return False
            x1,y1,dy1 = xm,ym,dym
    # Should have returned succes/fail before getting here
    raise RuntimeError("Maximum iterations exceeded: p=%s"%p)

# Iteratively check whether adjacent cells on boundary intersect hyperplane
# Add the ones that do to grow the crystal
Q = {seed}
allQ = set() 
for i in range(Ndepth):
    Q = {tuple(q) for p in Q for q in D+p if check_pm(q)} - allQ
    allQ |= Q
    print('\rGrowing crystal iteration %d/%d'%(i,Ndepth),' '*10,end='',flush=True)
print('\ndone')
figure(figsize=(10,10))
plotpoints(array(list(allQ)).T)

seed: (5770389, 10037825, 8425238, 1877260)


AssertionError: 